In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
import os
import torch
import numpy as np
import pickle 

# import nnfabrik
from nnfabrik import builder

# import nnvision
from numpy import linalg
import matplotlib.pyplot as plt
import seaborn as sns
from neuralpredictors.data.datasets import FileTreeDataset
# from neuralpredictors.data.transforms import Subsample, ToTensor, NeuroNormalizer, AddBehaviorAsChannels
from collections import OrderedDict
from cotton2020 import get_oracles, get_correlations

# Training Configuration

Configuration of the model, the trainer and the dataset used for training. 

In [8]:
scan = '20457-5-9'
paths = [f'data/static{scan}-preproc0/']


model_fn = 'model_components.se_core_spatialXfeature_readout'
model_config = {
   'pad_input': False,
   'stack': -1,
   'layers':4,
   'input_kern': 15,
   'gamma_input': 1., #20 #6.3831,          
   'gamma_readout':  0.002362354239446914, #0.0076,
   'hidden_dilation': 1,
   'hidden_kern': 13,
   'hidden_channels': 64,
   'n_se_blocks': 0,
   'depth_separable': True,
   'normalize': False,
    'init_noise' :4.1232e-05,
}

trainer_fn = 'cotton2020.nnvision_trainer'
trainer_config = dict(max_iter=100,
                      verbose=False, 
                      lr_decay_steps=4,
                      avg_loss=False, 
                      patience=5,
                      lr_init=.0041)

readout_key = f'{scan}-0/'

# Train Full Klindt et al. model

## Get DataLoader

In [9]:
unit_ids = np.load(f'configs/{scan}_train_units.npy')

dataset_fn = 'cotton2020.neurips_loaders'
dataset_config = dict(
    paths=paths,
    areas = ('V1',),
    batch_size=64,
    normalize=True,
    neuron_ids=[unit_ids],
)

dataloaders = builder.get_data(dataset_fn, dataset_config)
dataloaders

OrderedDict([('train',
              OrderedDict([('20457-5-9-0/',
                            <torch.utils.data.dataloader.DataLoader at 0x7f2fb903fa00>)])),
             ('validation',
              OrderedDict([('20457-5-9-0/',
                            <torch.utils.data.dataloader.DataLoader at 0x7f2fa5dd1700>)])),
             ('test',
              OrderedDict([('20457-5-9-0/',
                            <torch.utils.data.dataloader.DataLoader at 0x7f2fa5dd15b0>)]))])

## Define Model

In [10]:
model = builder.get_model(model_fn=model_fn, 
                          model_config=model_config, 
                          dataloaders=dataloaders, 
                          seed=1000)
model

Encoder(
  (core): SE2dCore(
    (_input_weights_regularizer): LaplaceL2norm(
      (laplace): Laplace()
    )
    (features): Sequential(
      (layer0): Sequential(
        (conv): Conv2d(1, 64, kernel_size=(15, 15), stride=(1, 1), bias=False)
        (norm): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        (nonlin): ELU(alpha=1.0, inplace=True)
      )
      (layer1): Sequential(
        (ds_conv): DepthSeparableConv2d(
          (in_depth_conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (spatial_conv): Conv2d(64, 64, kernel_size=(13, 13), stride=(1, 1), padding=(6, 6), groups=64, bias=False)
          (out_depth_conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (norm): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        (nonlin): ELU(alpha=1.0, inplace=True)
      )
      (layer2): Sequential(
        (ds_conv): DepthSeparableConv2d(
        

## Get Trainer

In [11]:
trainer = builder.get_trainer(trainer_fn, trainer_config)

## Run Training

In [13]:
score, output, model_state = trainer(model=model, dataloaders=dataloaders, seed=1000)

Epoch 1: 100%|██████████| 70/70 [00:12<00:00,  5.71it/s]


[001|00/05] ---> 0.06483694911003113


Epoch 2: 100%|██████████| 70/70 [00:07<00:00,  9.55it/s]


[002|00/05] ---> 0.09954481571912766


Epoch 3: 100%|██████████| 70/70 [00:07<00:00,  9.59it/s]


...

[067|03/05] -/-> 0.2673272490501404


Epoch 68: 100%|██████████| 70/70 [00:07<00:00,  9.28it/s]


[068|04/05] -/-> 0.2684917151927948


Epoch 69: 100%|██████████| 70/70 [00:07<00:00,  9.35it/s]


[069|05/05] -/-> 0.26971253752708435
Restoring best model after lr decay! 0.269713 ---> 0.271334
Restoring best model! 0.271334 ---> 0.271334


In [14]:
state_dict = model.state_dict()
with open(f'models/{scan}_full_model_statedict.pkl', 'wb') as fid:
    pickle.dump(state_dict, fid)

# Train Readout only

In [15]:
total_results = dict()

In [76]:
data_size = 1000


In [77]:
unit_ids = np.load(f'configs/{scan}_test_units.npy')
train_trials =  np.load(f'configs/{scan}_train_trials.npy')
dataset_fn = 'cotton2020.neurips_loaders'
dataset_config = dict(
    paths=paths,
    batch_size=64,
    normalize=True,
    neuron_ids=[unit_ids],
    trial_ids = dict(train=train_trials[:data_size]) if data_size <= 1000 else None
)
dataloaders = builder.get_data(dataset_fn, dataset_config)
dataloaders

Subsampling train set to 1000 trials


OrderedDict([('train',
              OrderedDict([('20457-5-9-0/',
                            <torch.utils.data.dataloader.DataLoader at 0x7f2fdc24d250>)])),
             ('validation',
              OrderedDict([('20457-5-9-0/',
                            <torch.utils.data.dataloader.DataLoader at 0x7f2f3c01a280>)])),
             ('test',
              OrderedDict([('20457-5-9-0/',
                            <torch.utils.data.dataloader.DataLoader at 0x7f2f34465610>)]))])

In [78]:
model = builder.get_model(model_fn=model_fn, 
                          model_config=model_config, 
                          dataloaders=dataloaders, 
                          seed=1000)
model

Encoder(
  (core): SE2dCore(
    (_input_weights_regularizer): LaplaceL2norm(
      (laplace): Laplace()
    )
    (features): Sequential(
      (layer0): Sequential(
        (conv): Conv2d(1, 64, kernel_size=(15, 15), stride=(1, 1), bias=False)
        (norm): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        (nonlin): ELU(alpha=1.0, inplace=True)
      )
      (layer1): Sequential(
        (ds_conv): DepthSeparableConv2d(
          (in_depth_conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (spatial_conv): Conv2d(64, 64, kernel_size=(13, 13), stride=(1, 1), padding=(6, 6), groups=64, bias=False)
          (out_depth_conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (norm): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        (nonlin): ELU(alpha=1.0, inplace=True)
      )
      (layer2): Sequential(
        (ds_conv): DepthSeparableConv2d(
        

## Load old model, copy core, and freeze it

In [79]:
with open(f'models/{scan}_full_model_statedict.pkl', 'rb') as fid:
    state_dict = pickle.load(fid)
    core_state_dict = OrderedDict(
        [(k[5:], v) for k,v in state_dict.items() if k.startswith('core.')]
    )
old_state = OrderedDict(core_state_dict) 
model.core.load_state_dict(core_state_dict)


for param in model.core.parameters():
    param.requires_grad = False

## Train

In [80]:
trainer = builder.get_trainer(trainer_fn, trainer_config)

In [ ]:
score, output, model_state = trainer(model=model, dataloaders=dataloaders, seed=1000)

Epoch 1:  62%|██████▎   | 10/16 [00:00<00:00, 10.66it/s]

## Make sure core was actually frozen

In [ ]:
for k, v in model.core.state_dict().items():
    if not 'running_mean' in k and not 'running_var' in k and not 'num_batches_tracked' in k:
        print(f'Checking {k}', (v - old_state[k]).abs().max())
        assert (v - old_state[k]).abs().max() < 1e-6
    

## Save model

In [ ]:
state_dict = model.state_dict()
with open(f'models/{scan}_datasize{data_size}readout_model_statedict.pkl', 'wb') as fid:
    pickle.dump(state_dict, fid)

## Evaluate model

In [ ]:
corrs = get_correlations(model, dataloaders['test'])
oracles = get_oracles(dataloaders["test"], as_dict=True)

In [ ]:
corrs.mean()

In [ ]:
a, _, _, _ = np.linalg.lstsq(oracles[readout_key][:, None], corrs)
print(f'Percent oracle is {a[0]*100:.2f}%')

In [ ]:
total_results[data_size] =  dict(
    unit_ids = dataloaders['train'][readout_key].dataset.neurons.unit_ids,
    test_correlation = corrs.squeeze(),
    oracle_correlation = oracles[readout_key],
    mean_test_correlation = corrs.mean(),
    fraction_oracle = a
)
np.savez(f'results/{scan}-data_efficieny.npz', total_results)

In [ ]:
{k:total_results[k]['mean_test_correlation'] for k in total_results}

Values from the Cotton et al. 2020